In [46]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
from itertools import combinations
import string
from collections import Counter, defaultdict

In [47]:
path_connected_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\"
path_temp_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\temporaryData\\"
year = "1999"

full_path = f"{path_connected_data}{year}_gptDataframe.csv"
grouped_path = f"{path_temp_data}{year}_grouped.csv"
word_path = f"{path_temp_data}institution_wordbank.csv"
ungrouped_path = f"{path_temp_data}{year}_unclassified_subinstitutions.csv"

high_confidence_path = f"{path_temp_data}{year}_high_confidence.csv"
medium_confidence_path = f"{path_temp_data}{year}_medium_confidence.csv"
low_confidence_path = f"{path_temp_data}{year}_low_confidence.csv"
nan_list_path = f"{path_temp_data}{year}_nan_confidence.csv"
and_keyword_path = f"{path_temp_data}{year}_and_title.csv"


pd.options.display.max_rows = 400

In [48]:
#define keywords
keywords = {
    "Arts and Sciences": ["arts and science", "letters and science", "arts, sciences, and letters", "arts, sciences", "arts & sciences", "letters, sciences", "arts and letters", "arts & letters"],
    "Engineering": ["engineering", "mines", "aerospace", "aeronautics"],
    "Applied Science": ["applied science"],
    
    "Computer Science": ["computer science", "computer,", "software engineering", "and computer", "computing"],
    "Data Science": ["data science", "artificial intelligence", "machine learning"],
    "Technology": ["technology"],
    "Information": ["information science", "information management", "information systems", "information stud", "library science", "and information", "library stud", "informatics", "school of information", 
                    "information and", "information school"],
    "Archictecure": ["architecture", "architectural", "urban design", "urban planning", "environmental design"],
    

    "Behavioral Science": ["psychology", "psychological", "human science", "human ecology", "behavioral", "human and", "childgood stud"],
    "Social Sciences": ["social science", "social and", "social ecology"],
    "Natural Sciences": ['natural science', "mathematics", "chemistry", "biology", "mathematical", "physical science", "life science", "biological science", 
                        "biochemistry", "natural and", "basic", "college of science", "school of science", ", science, and", ", science and", "physics"],
    "Environmental and Earth Science": ["environmental science", "forestry", "natural resource", "college of forest", 
                              "school of forest", "school of environment ", "environmental and", "geology", "oceanography", "earth science", "geophysics", "seismology", "volcanology", "earth and" 
                              "paleontology", "mineral science", "geoscience", "earth science", "earth studies", "atmospheric"],
    "Marine Science": ["marine science", "marine stud", "ocean science", "oceanography", "oceanographic", "ocean and"],

    
    "Business": ["business", "commerce", "mba", "college of economics", "and economics", "finance", "entrepreneurship", "school of management", "and management", "accounting", "agribusiness", "economic,", "accountancy"],
    "Hospitality": ["hospitality", "hotel administration", "hotel management", "restaurant management"],
    "Education": ["college of education", "school of education", "teaching", "teach", "curriculum development", "human development", "and education", ", education and"],
    "Criminal Justice": ["criminal justice", "criminology", "criminal", "forensics"],


    "Law": ["law", "legal studies"],
    "Policy and Affairs": ["public policy", "international policy", "public affairs", "international affairs", "environmental affairs", "urban affairs", "metropolitan affairs", "international service", 
                           "diplomacy", "diplomat", "international studies", "public administration", "urban policy", "global studies", "international relations", "social policy", "peace studies", "global management"],
    

    "Agriculture": ["agriculture", "agricultural", "agribusiness"],
    "Family and Consumer Science": ["family and", "home economics", "consumer science", "family science", "home"],
    "Culinary": ["culinary", "cooking", "gastronomy"],


    "Journalism": ["journalism"],
    "Communication": ["communication"],
    "Liberal Arts": ["liberal art", "liberal studies", "interdisciplinary studies", "general studies", "liberal and"],
    "Humanities": ["humanities", "history", "historical", "philosophy", "literature", "political"],
    "Fine Arts": ["fine arts", "music", "theater", "creative", "theatre", "film", "performing art", "the arts", "design art", "dramatic arts", "dance",
                   "visual arts", "school of drama", "recording arts", "ceramics", "fine and applied art", "school of design", "art and design", "fine", 
                   "cinema", "art ", "art, ", "studio art", "media art", "arts and performance", "drama", "arts and media"],
    "Language and Culture": ["languages", "lingusitics", "cultures", "translational studies", "intercultural studies", "ethnic studies", "hebraic studies", "cultural stud", "asian studies", "culture "],


    "Nursing": ["nursing"],
    "Physical Therapy and Nutrition": ["physical therapy", "nutrition", "physiotherapy", "nutritional therapy", "rehab", "human performance"],
    "Medical": ["medical", "pharmacy", "surgeons", "school of medicine", "college of medicine", "podiatric medicine", "osteopathic medicine"],
    "Pharmacy": ["pharmacy", "pharmacology"],
    "Dentistry" : ["dentistry", "dental"],
    "Optometry" : ["optometry", "optics", "optical science"],
    "Veterinary" : ['veterinary'],
    "Audiology": ["audiology"],
    "Gerontology": ["gerontology"],
    "Pathology": ["pathology"],
    "Health Science": ["health science", "public health", "health profession", "college of health", "school of health", "biomedical science", "and health", "allied health", "physical education",
                        "health and", "health studies", "genome"],


    "Religious Studies": ["religious", "theology", "divinity", "biblical", "theological", "religion", "seminary", "religous", "christ college"],
    "Social Work": ["social work", "social welfare", "community service", "human service", "social service", "public service"],
    

    "Graduate": ["graduate school", "graduate studies", "graduate college", "college of graduate", "school of graduate",  "advancing", "advanced study", "graduate division", "graduate and professional studies", "doctoral", "graduate study", "graduate "],
    "Continued Studies": ["extended studies", "extended education", "continuing education", "continued education", "continuing studies", "lifelong learning", "professional programs", "extended learning",
                           "adult education", "online learning", "adult student", "lifelong education", "continuing and", "university extension", "distance learning", "continuous education"],
    "Professional Studies": ["professional studies", "professional development", "professional education", "school of professions"],
    "Administration": ["academic service", "academics", "academic program", "admissions", "admission", "enrollment service", "student life", "housing", "academic support", "advising", "enrollment management", 
                       "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", 
                       "university librar", "information service", "information access", "library access", "residence", "academic affairs", "library affairs", "academic succes", "first year student",
                       "undergraduate studies", "undergraduate education"],
    "Honor College": ["honor college", "honors college", "scholar program"],

    "Satellite Campus": ["campus"],
    "Foundation": ["foundation"],
    "Boards": ["board of", "regents", "trustees", "council of"],
    "Throwaway": ["placeholder value"],
    "Hospital": ["hospital"],
    "University College": ["university college"]
}

In [49]:
keyword2category = {k:c for c, ks in keywords.items() for k in ks}
#dictionary of "school of" + category and #college of" + category
schoolwords = {"school of "+s.lower():s for s in keywords.keys()}
schoolwords.update({"college of "+s.lower():s for s in keywords.keys()})
schoolwords['school of medicine'] = "Medical"
schoolwords['school of management'] = "Business"
schoolwords['college of management'] = "Business"
schoolwords['school of economics'] = "Business"
schoolwords['teachers college'] = "Education"
schoolwords["college of science"] = "Natural Sciences"
schoolwords["school of science"] = "Natural Sciences"
schoolwords["college of sciences"] = "Natural Sciences"
schoolwords["school of sciences"] = "Natural Sciences"
schoolwords["college of design"] = "Fine Arts"
schoolwords["school of design"] = "Fine Arts"
schoolwords["college of art"] = "Fine Arts"
schoolwords["school of art"] = "Fine Arts"
schoolwords["college of arts"] = "Fine Arts"
schoolwords["school of arts"] = "Fine Arts"
schoolwords["library"] = "Administration"
schoolwords["libraries"] = "Administration"
schoolwords["students"]  = "Administration"
schoolwords["university library"] = "Administration"

In [50]:
def parse_columns(path, size):
    df = pd.read_csv(path)
    all_columns = df.iloc[:, :size]
    return all_columns

fulldf = parse_columns(full_path, 8)

In [51]:
#create table of unique institution + subinstitutions
unique_combinations = set()
all_subinstitutions = []
for index, row in fulldf.iterrows():
    institution = row["Institution"]
    subinstitution = row["SubInstitution"]
    combination = (institution, subinstitution)
    if combination not in unique_combinations and pd.notnull(subinstitution):
        unique_combinations.add(combination)
        all_subinstitutions.append(combination)

unique_df = pd.DataFrame(all_subinstitutions, columns=["Institution", "SubInstitution"])
unique_df["Category"] = ""
print(unique_df.shape)

(3630, 3)


In [52]:
#now find the high-confidence classifcations. e.g. "school of + category"
#Checking for equality, not substring, also doesnt fall under multiple categories
unique_df["Category"] = ""
high_confidence = pd.DataFrame(columns=["Institution", "SubInstitution", "Category"])
uncategorized_df = pd.DataFrame(columns=["Institution", "SubInstitution", "Category"])

for index, row in unique_df.iterrows():
    subinstitution = row["SubInstitution"].lower()
    for key, value in schoolwords.items():
        if key == subinstitution:
            if(row["Category"] == ""):
                unique_df.at[index, "Category"] += str(value)
                
            else:
                unique_df.at[index, "Category"] += str(value) + ", "
    if unique_df.at[index, "Category"] != "":
        high_confidence = pd.concat([high_confidence, unique_df.loc[[index]]], ignore_index=True)
    else:
        uncategorized_df = pd.concat([uncategorized_df, unique_df.loc[[index]]], ignore_index=True)

# high_confidence.to_csv(high_confidence_path, index = False)
print(high_confidence.shape)


(1351, 3)


In [53]:
#count what category each of the high-confidence classifications are in
category_counter = Counter()
for category_list in high_confidence["Category"]:
    categories = [cat.strip() for cat in category_list.split(",") if cat.strip()]
    category_counter.update(categories)

total_categories = sum(category_counter.values())
category_fractions = {category: count / total_categories for category, count in category_counter.items()}
sorted_category_fractions = dict(sorted(category_fractions.items(), key=lambda item: item[1], reverse=True))

print("\nPercentage of each category: (high conf):")
for category, fraction in sorted_category_fractions.items():
    print(f"{category}: {fraction:.4f}")


Percentage of each category: (high conf):
Education: 0.1732
Arts and Sciences: 0.1532
Nursing: 0.0977
Engineering: 0.0910
Business: 0.0873
Law: 0.0755
Liberal Arts: 0.0392
Natural Sciences: 0.0385
Medical: 0.0370
Pharmacy: 0.0340
Social Work: 0.0289
Fine Arts: 0.0281
Dentistry: 0.0259
Agriculture: 0.0133
Administration: 0.0126
Professional Studies: 0.0118
Technology: 0.0104
Social Sciences: 0.0089
Communication: 0.0081
Humanities: 0.0067
Optometry: 0.0059
Criminal Justice: 0.0044
Journalism: 0.0037
Computer Science: 0.0015
Marine Science: 0.0007
Applied Science: 0.0007
Information: 0.0007
Gerontology: 0.0007


In [54]:
#categorize rest of the subinstitutions, including those who fall under multiple categories
medium_confidence = pd.DataFrame(columns=["Institution", "SubInstitution", "Category"])
uncategorized_df["Category"] = ""

for index, row in uncategorized_df.iterrows():
    institution = row["Institution"].title()
    subinstitution = row["SubInstitution"].title()
    #check every word in the subinstitution
    matched = False
    for key, value in keyword2category.items():
        # print(word, key)
        if key.title() in subinstitution:
            if not matched:
                uncategorized_df.at[index, "Category"] += str(value)
                matched = True
            elif matched and str(value) not in uncategorized_df.at[index, "Category"]:
                uncategorized_df.at[index, "Category"] += ", " + str(value)
    if uncategorized_df.at[index, "Category"] != "":
        medium_confidence = pd.concat([medium_confidence, uncategorized_df.loc[[index]]], ignore_index=True)
        unique_df.loc[unique_df['SubInstitution'].str.title() == subinstitution, 'Category'] = uncategorized_df.at[index, 'Category']
    elif institution in subinstitution and uncategorized_df.at[index, "Category"] == "":
        uncategorized_df.at[index, "Category"] = "Satellite Campus"
        medium_confidence = pd.concat([medium_confidence, uncategorized_df.loc[[index]]], ignore_index=True)
        unique_df.loc[unique_df['SubInstitution'].str.title() == subinstitution, 'Category'] = uncategorized_df.at[index, 'Category']

# medium_confidence.to_csv(medium_confidence_path, index = False)
# unique_df.to_csv(low_confidence_path, index = False)

In [55]:
# arts and science & fine arts being categorized together but shouldnt
#administration cannot combine with others
for index, row in medium_confidence.iterrows():
    category = row["Category"]
    subinstitution = row["SubInstitution"].title()
    institution = row["Institution"].title()
    word_list = list(subinstitution.split())
    art_count = word_list.count("Arts")
    engineering_count = word_list.count("Engineering")
    #if "Humanities" in category and "Social Sciences" in category and "Arts" in subinstitution and "Fine Arts" not in subinstitution:
    #     medium_confidence.at[index, "Category"] += ", " + "Arts and Sciences"
    if "Humanities" in category and "Arts and Sciences" in category and "Fine Arts" in category and "Fine Arts" in subinstitution and art_count == 1:
        medium_confidence.at[index, "Category"] = category.replace("Arts and Sciences", "").strip(", ")
    if "Communication Arts And Science" in subinstitution and "Arts and Sciences" in category:
        medium_confidence.at[index, "Category"] = category.replace("Arts and Sciences", "").strip(", ")
    if "Arts and Sciences" not in category and "Fine Arts" not in category and "Liberal Art" not in subinstitution and art_count == 1:
         medium_confidence.at[index, "Category"] += ", Fine Arts"
    if "Administration" in category:
        medium_confidence.at[index, "Category"] = "Administration"
    if "Arts," in subinstitution and "Arts And Sciences" not in category and "Liberal Arts" not in category and "Fine Arts" not in category:
        medium_confidence.at[index, "Category"] += ", Fine Arts"
    if "Fine Arts" in category and "Liberal Arts" in category and art_count == 1:
        medium_confidence.at[index, "Category"] = category.replace("Fine Arts", "").strip(", ")
    if "And Science" in subinstitution and "Arts and Sciences" not in category and "Natural Sciences" not in category:
        medium_confidence.at[index, "Category"] += ", Natural Sciences"
    if "Liberal Arts" in category and "Arts and Sciences" in category:
        medium_confidence.at[index, "Category"] = category.replace("Liberal Arts", "").strip(", ")
    # if "Engineering" in category and "Applied Science" in category:
    #     medium_confidence.at[index, "Category"] = category.replace("Applied Science", "").strip(", ")
    if "Software Engineering" in subinstitution and "Engineering" in category and engineering_count == 1:
        medium_confidence.at[index, "Category"] = category.replace("Engineering", "").strip(", ")
    if "Hospitality" in subinstitution and "Hospital" in category:
        medium_confidence.at[index, "Category"] = category.replace("Hospital,", "").strip(", ")

In [56]:
#categorize remaining "others"
uncategorized_df["Category"] = ""
na_df = pd.DataFrame(columns=["Institution", "SubInstitution", "Category"])
for index, row in unique_df.iterrows():
    if row["Category"] == "":
        na_df = pd.concat([na_df, unique_df.loc[[index]]], ignore_index = True)

na_df["Category"] = "Other"
    
institution_list = list(set(fulldf["Institution"].tolist()))
print(institution_list)
for index, row in na_df.iterrows():
    category = row["Category"]
    subinstitution = row["SubInstitution"].title()
    institution = row["Institution"].title()
    word_list = list(subinstitution.split())
    institution_appears = any(substring in subinstitution for substring in institution_list)
    if institution_appears:
        na_df.at[index, "Category"] = "Throwaway"
    if "Library" in subinstitution:
        na_df.at[index, "Category"] = "Administration"
na_df.to_csv(ungrouped_path)

['Barry University', 'University Of Richmond', 'University Of Southwestern Louisiana', 'University Of Detroit Mercy', 'Birmingham Southern College', 'California State University Long Beach', 'Northeastern University', 'East Carolina University', 'Kennesaw State University', 'University Of Maine At Presque Isle', 'Gettysburg College', 'Westminster College', 'University System Of Maryland', 'University Of Wisconsin River Falls', 'Hofstra University', 'Truman State University', 'Linfield College', 'Union Institute', 'Brandeis University', 'St Olaf College', 'Wheaton College-Illinois', 'Winthrop University', 'San Francisco State University', 'St Lawrence University', 'Oklahoma State University', 'Bradley University', 'Texas A&M University College Station', 'College Of Mount St Joseph', 'Ohio Wesleyan University', 'Tennessee State University', 'Elizabeth City State University', 'Lamar State College–Orange', 'University Of North Dakota', 'Oregon University System', 'University Of Texas Healt

In [57]:
#count how much each category appears for the medium confidence 
category_counter = Counter()
for category_list in medium_confidence["Category"]:
    categories = [cat.strip() for cat in category_list.split(",") if cat.strip()]
    category_counter.update(categories)


total_categories = sum(category_counter.values())
category_fractions = {category: count / total_categories for category, count in category_counter.items()}
sorted_category_fractions = dict(sorted(category_fractions.items(), key=lambda item: item[1], reverse=True))

print("\nCategory Percentages (all classified):")
for category, fraction in sorted_category_fractions.items():
    print(f"{category}: {fraction:.4f}")


Category Percentages (all classified):
Business: 0.1137
Graduate: 0.1100
Fine Arts: 0.0705
Health Science: 0.0705
Natural Sciences: 0.0454
Engineering: 0.0421
Education: 0.0410
Arts and Sciences: 0.0395
Social Work: 0.0277
Archictecure: 0.0262
Policy and Affairs: 0.0255
Administration: 0.0255
Technology: 0.0255
Humanities: 0.0236
Medical: 0.0236
Social Sciences: 0.0218
Environmental and Earth Science: 0.0210
Agriculture: 0.0196
Behavioral Science: 0.0196
Communication: 0.0181
Satellite Campus: 0.0177
Law: 0.0166
Applied Science: 0.0162
Information: 0.0151
Nursing: 0.0140
Religious Studies: 0.0122
Continued Studies: 0.0114
Veterinary: 0.0092
Liberal Arts: 0.0092
University College: 0.0085
Professional Studies: 0.0070
Computer Science: 0.0070
Honor College: 0.0063
Physical Therapy and Nutrition: 0.0059
Journalism: 0.0059
Pharmacy: 0.0055
Dentistry: 0.0052
Family and Consumer Science: 0.0052
Boards: 0.0033
Hospitality: 0.0030
Marine Science: 0.0022
Language and Culture: 0.0015
Hospital: 

In [58]:
combination_counter = Counter()
subinstitution_texts = {}

for index, row in medium_confidence.iterrows():
    category_list = row["Category"]
    subinstitution = row["SubInstitution"]
    categories = sorted([cat.strip() for cat in category_list.split(",") if cat.strip()])

    if len(categories) > 1:
        for size in range(2, len(categories) + 1):
            for c in combinations(categories, size):
                #convert combination to a tuple to use it as dict key
                c_tuple = tuple(c)
                combination_counter[c_tuple] += 1
                
                if c_tuple not in subinstitution_texts:
                    subinstitution_texts[c_tuple] = []
                subinstitution_texts[c_tuple].append(subinstitution)

sorted_combinations = sorted(combination_counter.items(), key=lambda item: item[1], reverse=True)

#This does not print combinations correctly. dont use as reference to gauge accuracy of classifications
print("Category combinations:")
for combination, count in sorted_combinations:
    print(f"{combination}: {count}")
    print("SubInstitutions:", subinstitution_texts[combination])
    print()

Category combinations:
('Fine Arts', 'Humanities'): 36
SubInstitutions: ['College of Humanities and Fine Arts', 'School of Arts and Humanities', 'College of Arts, Humanities and Social Sciences', 'College of Humanities and the Arts', 'School of Arts and Humanities', 'College of Arts and Humanities', 'School of Arts and Humanities', 'School of Arts and Humanities', 'College of Architecture, Arts and Humanities', 'College of Arts and Humanities', 'School of Arts and Humanities', 'College of Arts and Humanities', 'College of Arts and Humanities', 'College of Arts, Humanities and Social Sciences', 'Arts and Humanities', 'College of Arts and Humanities', 'College of Arts and Humanities', 'College of Fine Arts and Humanities', 'School of Arts and Humanities', 'College of Arts, Humanities and Social Sciences', 'College of Humanities, Arts and Social Sciences', 'College of Arts and Humanities', 'College of Humanities, Fine Arts and Communication', 'The College of Humanities and Fine Arts', 'Co

In [59]:
#number of elements that have yet to be categorized
items_left = (unique_df.shape)[0]  - (high_confidence.shape)[0] - (medium_confidence.shape)[0]
print(100*(items_left/(unique_df.shape)[0]), "%")

5.702479338842975 %


In [60]:
# uncategorized_df["Category"] = ""
# na_df = pd.DataFrame(columns=["Institution", "SubInstitution", "Category"])
# for index, row in unique_df.iterrows():
#     if row["Category"] == "":
#         na_df = pd.concat([na_df, unique_df.loc[[index]]], ignore_index = True)

# na_df["Category"] = "Other"
# na_df.to_csv(nan_list_path, index = False)

In [61]:
# Combine the DataFrames
combined_df = pd.concat([medium_confidence, na_df, high_confidence], ignore_index=True)
sorted_combined_df = combined_df.sort_values(by="Category")
sorted_combined_df.to_csv(grouped_path, index=False)

In [62]:
category_counter = Counter()
for category_list in combined_df["Category"]:
    categories = [cat.strip() for cat in category_list.split(",") if cat.strip()]
    category_counter.update(categories)

sorted_category = dict(sorted(category_counter.items(), key=lambda item: item[1], reverse=True))
print("\nCategory counts")
for category, number in sorted_category.items():
    print(f"{category}: {number}")


Category counts
Business: 426
Education: 345
Arts and Sciences: 314
Graduate: 298
Engineering: 237
Fine Arts: 229
Health Science: 191
Other: 191
Natural Sciences: 175
Nursing: 170
Law: 147
Social Work: 114
Medical: 114
Administration: 93
Technology: 83
Liberal Arts: 78
Humanities: 73
Archictecure: 71
Agriculture: 71
Social Sciences: 71
Policy and Affairs: 69
Pharmacy: 61
Communication: 60
Environmental and Earth Science: 57
Behavioral Science: 53
Dentistry: 49
Satellite Campus: 48
Applied Science: 45
Information: 42
Professional Studies: 35
Religious Studies: 33
Continued Studies: 31
Veterinary: 25
University College: 23
Computer Science: 21
Journalism: 21
Honor College: 17
Physical Therapy and Nutrition: 16
Family and Consumer Science: 14
Boards: 9
Throwaway: 9
Hospitality: 8
Optometry: 8
Marine Science: 7
Criminal Justice: 7
Language and Culture: 4
Hospital: 1
Culinary: 1
Audiology: 1
Pathology: 1
Gerontology: 1


In [63]:
#rows with "and" but only classified under 1 category
and_df = []
for index, row in combined_df.iterrows():
    position = row["SubInstitution"]
    category = row["Category"]
    if "And" in position.title() and "," not in category:
        and_df.append(row)
and_df = pd.DataFrame(and_df)
# and_df.to_csv(and_keyword_path, index = False)

In [64]:
#things not classified as education but has education in
for index, row in sorted_combined_df.iterrows():
    subinstitution = row["SubInstitution"].title()
    category = row["Category"].title()
    if "Education" in subinstitution and "Education" not in category and "Continued Studies" not in category:
        print(subinstitution)

Undergraduate Education
Undergraduate Education
College Of Health, Physical Education, Recreation And Dance
Azrieli Graduate School Of Jewish Education And Administration
College Of Health, Physical Education And Recreation
School Of Physical Education
School Of Health, Physical Education And Recreation
School Of Physical Education
School Of Physical Education And Athletic Department
Sophie Davis School Of Biomedical Education/Cuny Medical School
School Of Graduate Medical Education
International Education
Abraham S. Fischler Center For Advancement Of Education
Educational Outreach
School Of Community Education
School Of Career Education
College Of Technical And Community Education
Education, Health And Rehabilitation


In [65]:
for index, row in sorted_combined_df.iterrows():
    subinstitution = row["SubInstitution"].title()
    category = row["Category"].title()
    if "Art" in subinstitution and "Arts And Sciences" not in category and "Liberal Arts" not in category and "Fine Arts" not in category:
        print(subinstitution)

Arthur J. Kania School Of Management
College Of Communication Arts And Sciences
Tyler School Of Art
Herron School Of Art
School Of Arts And Professions
Lamar University-Port Arthur


In [66]:
for index, row in sorted_combined_df.iterrows():
    subinstitution = row["SubInstitution"].title()
    category = row["Category"].title()
    if "And Science" in subinstitution and "Natural Sciences" not in category and "Arts And Sciences" not in category:
        print(subinstitution)
    # if "Health And" in subinstitution:
    #     print(subinstitution)

College Of Communication Arts And Sciences


In [67]:
for index, row in sorted_combined_df.iterrows():
    subinstitution = row["SubInstitution"].title()
    category = row["Category"].title()
    for key in keywords.keys():
        if key in subinstitution and key not in category:
            print(key, ": ", subinstitution)

Education :  Undergraduate Education
Technology :  Instructional Technology And Academic Computing
Information :  Library And Information Services
Information :  Information Services And Technologies
Education :  Undergraduate Education
Liberal Arts :  St. John'S College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Arts :  School Of Liberal Arts And Sciences
Liberal Arts :  College Of Liberal Arts And Sciences
Liberal Art